In [20]:
import pandas as pd
import errno    
import os
import numpy as np
import math
import time

'''
Author: Enrico Ceccolini
    Create a dataframe to be used as the base of a power prediction model
'''

datadir = "/datasets/eurora_data/db1/"
datadir = "/datasets/eurora_data/db/"
datadir = "/datasets/eurora_data/db_local/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-11-01')
#train_start_time = pd.to_datetime('2014-03-31')
#train_end_time = pd.to_datetime('2014-10-01')
#test_end_time = pd.to_datetime('2014-11-01')

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"




#infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset"
infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset"
outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"

#infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset_clear"
#infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset_clear"
#outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_cleaned"

#### random selection
#infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset_random"
#infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset_random"
#outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_random"
#infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_random"
#outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_random"


nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [21]:
jobs_train = pd.read_csv(infile_jobs_trainset + ".csv", index_col=0)
print("train set contains {} records".format(jobs_train.shape[0]))
jobs_test = pd.read_csv(infile_jobs_testset + ".csv", index_col=0)
print("test set contains {} records".format(jobs_test.shape[0]))

train set contains 33541 records
test set contains 1611 records


In [22]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records
after the clean, jobs_to_nodes_whole_data contains 444610 records


In [23]:
###### NB eseguire solo se non si vogliono job sotto 5 sec

#jobs_train = jobs_train[pd.to_datetime(jobs_train['end_time']) - pd.to_datetime(jobs_train['run_start_time']) > np.timedelta64(5, 's')]
#print(jobs_train.shape[0])
#jobs_test = jobs_test[pd.to_datetime(jobs_test['end_time']) - pd.to_datetime(jobs_test['run_start_time']) > np.timedelta64(5, 's')]
#print(jobs_test.shape[0])

In [24]:
# train_whole: 167324 ; test_whole:29935 
# train_long: 39363 ; test_long: 11622

In [25]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_train, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 39150 records


In [26]:
jobs_to_nodes_whole_data.head(1)

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested
job2node_id,,,,,,
1,17,498458.node129,16,0,0,14680064


In [27]:
jobs_train.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,n_2_2,n_3_1,pred_overAll,pred_perNode,n_cores_2_1,n_cores_2_2,n_cores_3_1,user_short,pred_user,pred_userJob
job_id,,,,,,,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,1.0,0.0,251.390328,292.871252,0.0,16.0,0.0,alupi000,225.932207,198.527435


### good code

We want to obtain 5 dataframes:
- node
- user
- user_node
- user_job
- user_job_node

In [28]:
merged_jobs_to_nodes['core_avg'] = merged_jobs_to_nodes['real_pow'] / merged_jobs_to_nodes['cpu_req']

In [29]:
start_time = time.time()
# user_jobName_node
df_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print("--- %s seconds ---" % (time.time() - start_time))
print(df_user_job_node.shape[0])

# Drop a row by condition
df_user_job_node = df_user_job_node[df_user_job_node['mean_core'] != 0.0]
df_user_job_node.head(100)

--- 0.040319204330444336 seconds ---
15345


,user,job_name,node_id,mean_core,count
0,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,34,7.842319,4
1,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,35,7.842319,4
2,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,36,7.842319,4
3,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,37,7.842319,4
4,a07smr01@node034.eurora.cineca.it,A2REPO,33,6.436153,3
5,a07smr01@node034.eurora.cineca.it,ARKI,33,5.768123,7
6,a07smr01@node034.eurora.cineca.it,LM28_OPE,34,15.002180,6
7,a07smr01@node034.eurora.cineca.it,LM28_OPE,35,15.002180,6
8,a07smr01@node034.eurora.cineca.it,LM28_OPE,36,15.002180,6
9,a07smr01@node034.eurora.cineca.it,LM28_OPE,37,15.002180,6


In [30]:
#outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
#outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_long"
df_user_job_node.to_csv(outfile_user_job_node + ".csv")

In [31]:
jobs_train.shape

(33541, 31)

# Convert to python dictionary

In [32]:
import pandas as pd
import errno    
import os

#infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
#infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_andrea_dataset"
user_job_node = pd.read_csv(infile_user_job_node + ".csv", index_col=0)
user_job_node.head(5)

,user,job_name,node_id,mean_core,count
0,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,34,7.842319,4
1,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,35,7.842319,4
2,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,36,7.842319,4
3,a07smr01@node033.eurora.cineca.it,SWAN_MED_ITA,37,7.842319,4
4,a07smr01@node034.eurora.cineca.it,A2REPO,33,6.436153,3


In [33]:
# creo il dizionario applicando raggruppamenti
my_dict = user_job_node.groupby(['user'])[['job_name', 'node_id', 'mean_core', 'count']].apply(lambda g: g.groupby(['job_name'])[['node_id', 'mean_core', 'count']].apply(lambda g1: g1.groupby(['node_id'])[['mean_core', 'count']].apply(lambda g2: g2.values.tolist()[0]).to_dict()).to_dict()).to_dict()
#my_dict

In [34]:
# aggiungo al dizionario i metodi che prevedono valori medi

user_dict = my_dict

for user_key, user_values in user_dict.items():
    user_job_dict = user_dict[user_key]
    for user_job_key, user_job_values in user_job_dict.items():
        if(user_key != 'node_avg' and user_job_key != 'user_node_avg' and user_job_key != 'user_avg'): # [user-job]
            
            mean_core_user_job = df_user_job_node.loc[(df_user_job_node['user']==user_key) & (df_user_job_node['job_name'] == user_job_key)]
            if(mean_core_user_job.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
                user_job_mean_core_wholeNodes = np.average(mean_core_user_job['mean_core'], weights=mean_core_user_job['count'])
                user_job_counts = mean_core_user_job['count'].sum()
                user_dict[user_key][user_job_key]['user_job_avg'] = list([user_job_mean_core_wholeNodes, user_job_counts])
    
    mean_core_user = df_user_job_node.loc[(df_user_job_node['user']==user_key)]
    if(mean_core_user.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        user_mean_core_wholeNodes = np.average(mean_core_user['mean_core'], weights=mean_core_user['count'])
        user_counts = mean_core_user['count'].sum()
        user_dict[user_key]['user_avg'] = list([user_mean_core_wholeNodes, user_counts])
        
        
    user_dict[user_key]['user_node_avg'] = {}
    for i in range(1, 65):
        mean_core_user_node = df_user_job_node.loc[(df_user_job_node['user']==user_key) & (df_user_job_node['node_id'] == i)]
        if(mean_core_user_node.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
            user_node_mean_core_wholeNodes = np.average(mean_core_user_node['mean_core'], weights=mean_core_user_node['count'])
            user_node_counts = mean_core_user_node['count'].sum()
            user_dict[user_key]['user_node_avg'][i] = list([user_node_mean_core_wholeNodes, user_node_counts])
    
user_dict['node_avg'] = {}
for i in range(1, 65):
    mean_core_node = df_user_job_node.loc[(df_user_job_node['node_id'] == i)]
    if(mean_core_node.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        node_mean_core_wholeNodes = np.average(mean_core_node['mean_core'], weights=mean_core_node['count'])
        node_counts = mean_core_node['count'].sum()
        user_dict['node_avg'][i] = list([node_mean_core_wholeNodes, node_counts])


In [35]:
import pickle

#outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
#outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_cleaned"

with open(outfile_user_job_node_dict + '.pkl', 'wb') as handle:
    pickle.dump(user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(outfile_user_job_node_dict + '.pkl', 'rb') as handle:
    b = pickle.load(handle)

print(user_dict == b)

True


In [36]:
print(outfile_user_job_node_dict)

/datasets/eurora_data/db_local/CPUs/Andrea/Andrea_mean_core_user_job_node_whole
